This notebook helps to populate the table from each sheet of MorpaData excel 





---


In [ ]:
%pip install --upgrade pip

In [ ]:
#check if numpy is installed

import subprocess

try:
    subprocess.check_call(['pip', 'show', 'numpy'])
except subprocess.CalledProcessError:
    # numpy is not installed
    %pip install numpy
else:
    print('numpy is already installed.')

In [ ]:
#check if openpyxl is installed

import subprocess

try:
    subprocess.check_call(['pip', 'show', 'openpyxl'])
except subprocess.CalledProcessError:
    # openpyxl is not installed
    %pip install openpyxl
else:
    print('openpyxl is already installed.')

In [ ]:
#check if pandas is installed

import subprocess

try:
    subprocess.check_call(['pip', 'show', 'pandas'])
except subprocess.CalledProcessError:
    # pandas is not installed
    %pip install pandas
else:
    print('pandas is already installed.')


In [ ]:
import pandas as pd
import numpy as np
import openpyxl

# Set display options to show all rows and columns
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
# input excel file path
inputExcelFile ="MorpaData.xlsx"

# creating or loading an excel workbook
newWorkbook = openpyxl.load_workbook(inputExcelFile)

# printing all the sheetnames in an excel file using sheetnames attribute
print('The Sheet names of the given excel file: \n')

# Getting the sheetnames as a list using the sheetnames attribute
sheetNames=newWorkbook.sheetnames

# Traversing in the sheetNames list
for name in sheetNames:
   print(name)

In [ ]:
# get the member data from excel

member_df = pd.read_excel('MorpaData.xlsx',sheet_name='Üye Listesi')
print(member_df.head(10),'\n\n')

print(member_df.info())

# get the unique number of members

print(member_df['Üye'].nunique())

In [ ]:
# get the Login logs data from excel

login_df = pd.read_excel('MorpaData.xlsx',sheet_name='Giriş Logları')
login_df['Giriş Zamanı'] = pd.to_datetime(login_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
login_df['Çıkış Zamanı'] = pd.to_datetime(login_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')
print(login_df.head(10),'\n\n')

print(login_df.info(),'\n\n')


# get the unique number of members on login data
print("unique member count on login data: ",login_df['Üye'].nunique())

In [ ]:
# get the lecturing logs data from excel

lecturing_df = pd.read_excel('MorpaData.xlsx',sheet_name='Konu anlatımı')
lecturing_df['Giriş Zamanı'] = pd.to_datetime(lecturing_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
lecturing_df['Çıkış Zamanı'] = pd.to_datetime(lecturing_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')

# calculate the duration of the lecturing

lecturing_df['lec_Süre'] = (lecturing_df['Çıkış Zamanı'] - lecturing_df['Giriş Zamanı']).dt.total_seconds().astype(int)

print(lecturing_df.head(10),'\n\n')

print(lecturing_df.info(),'\n\n')


# get the unique number of members on lecturing data
print("Unique number of users on lecturing: ", lecturing_df['Üye'].nunique())

In [ ]:
# get the studies data from excel

studies_df = pd.read_excel('MorpaData.xlsx',sheet_name='çalışmalar')
studies_df['Giriş Zamanı'] = pd.to_datetime(studies_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
studies_df['Çıkış Zamanı'] = pd.to_datetime(studies_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')

studies_df['stud_Süre'] = (studies_df['Çıkış Zamanı'] - studies_df['Giriş Zamanı']).dt.total_seconds().astype(int)

print(studies_df.head(10),'\n\n')

print(studies_df.info(),'\n\n')

# get the unique number of members on studies data
print("unique member count on studies data: ",studies_df['Üye'].nunique())

In [ ]:
# get the exams data from excel

exams_df = pd.read_excel('MorpaData.xlsx',sheet_name='Sınav')
exams_df['Giriş Zamanı'] = exams_df['Giriş Zamanı'].apply(lambda x: x.strftime("%d/%m/%Y  %H:%M:%S"))
exams_df['Çıkış Zamanı'] = exams_df['Çıkış Zamanı'].apply(lambda x: x.strftime("%d/%m/%Y  %H:%M:%S"))

exams_df['Giriş Zamanı'] = pd.to_datetime(exams_df['Giriş Zamanı'], format='%d/%m/%Y %H:%M:%S')
exams_df['Çıkış Zamanı'] = pd.to_datetime(exams_df['Çıkış Zamanı'], format='%d/%m/%Y %H:%M:%S')

exams_df['Giriş Zamanı']

print(exams_df.head(10),'\n\n')

print(exams_df.info(),'\n\n')

# get the unique number of members on exams data
print("unique member count on exams data: ",exams_df['Üye'].nunique())


# wrong answers are not deducted from the total score.

In [ ]:
# get the subject data from excel

subject_df = pd.read_excel('MorpaData.xlsx',sheet_name='konu')
subject_df.head(10)

#sort according to aktif materyal sayısı

subject_df.sort_values(by=['Aktif Materyal Sayısı'],ascending=False)


In [ ]:
#left join the exam and subject dataframes on Konu column

exam_subject_df = pd.merge(exams_df,subject_df,how='left',left_on='Konu',right_on='Konu')

# change column names for further join operations. Sınav related columns are renamed with S_ prefix
exam_subject_df.rename(
    columns={"Giriş Zamanı": "S_Giriş Zamanı", "Çıkış Zamanı": "S_Çıkış Zamanı", "Süre":"S_süre"},
    inplace=True,
)


In [ ]:
# drop non-numeric sınıf rows

exam_subject_df = exam_subject_df[exam_subject_df['Sınıf'].notna()]

exam_subject_df


# check data type of exam subject dataframe

exam_subject_df.info()

In [ ]:
# change data type to int which are not int

exam_subject_df['Sınıf'] = exam_subject_df['Sınıf'].astype('int64')
exam_subject_df['Ders'] = exam_subject_df['Ders'].astype('int64')
exam_subject_df['Aktif Materyal Sayısı'] = exam_subject_df['Aktif Materyal Sayısı'].astype('int64')
exam_subject_df['Toplam Materyal Sayısı'] = exam_subject_df['Toplam Materyal Sayısı'].astype('int64')

exam_subject_df.head(10)



Alttaki kodu anlamadım. Tekrar incele

In [ ]:
# add cumulative lecture time to exam_subject dataframe for each member according to the subject prior to exam date s_Giriş Zamanı

# calculate cumulative lecture time before the exam date for each member of each subject
exam_subject_df['cum_lecture_time'] = 0
for index, row in exam_subject_df.iterrows():
    df = lecturing_df[lecturing_df['Üye'] == row['Üye']] #Filters the lecturing_df DataFrame to select rows where the 'Üye' column matches the current row's 'Üye' value in exam_subject_df
    df = df[df['Konu'] == row['Konu']] #Further filters the df DataFrame to select rows where the 'Konu' column matches the current row's 'Konu' value in exam_subject_df
    df = df[df['Giriş Zamanı'] < row['S_Giriş Zamanı']] #Further filters the df DataFrame to select rows where the 'Giriş Zamanı' column is less than the current row's 'S_Giriş Zamanı' value in exam_subject_df. This ensures that only lecture entries that occurred before the exam start time are considered.
    df['lec_Süre'] = df['lec_Süre'].astype('int64')
    exam_subject_df.at[index,'cum_lecture_time'] = df['lec_Süre'].sum()

# calculate cumulative study time before the exam date for each member of each subject
exam_subject_df['cum_study_time'] = 0
for index, row in exam_subject_df.iterrows():
    df = studies_df[studies_df['Üye'] == row['Üye']]
    df = df[df['Konu'] == row['Konu']]
    df = df[df['Giriş Zamanı'] < row['S_Giriş Zamanı']]
    df['stud_Süre'] = df['stud_Süre'].astype('int64')
    exam_subject_df.at[index,'cum_study_time'] = df['stud_Süre'].sum()

exam_subject_df['total_study_time'] = exam_subject_df['cum_lecture_time'] + exam_subject_df['cum_study_time']
        
exam_subject_studies_df = exam_subject_df

In [ ]:
# to be properly label, do necessary sorting operations on exam_subject_studies_df

exam_subject_studies_df.sort_values(by=['Üye','Sınav','Konu'])


In [ ]:
# to tag the members who attempt gaming the system, I crate a deep copy of exams_subject_studies df_gtsLabeled.

df_gtsLabeled = exam_subject_studies_df.copy(deep=True)

# create a new column for labeling and overlapping time

df_gtsLabeled['Label'] = ""
df_gtsLabeled['Overlap (secs.)'] = ""

# update the index of df_gtsLabeled

df_gtsLabeled.reset_index(drop=True, inplace=True)

# then we write the conditions for labeling the data who "Gaming the System". 1 for gaming, 0 for not gaming.

for i in range(1,len(df_gtsLabeled)):
    if df_gtsLabeled['Üye'][i-1] == df_gtsLabeled['Üye'][i]:
        if df_gtsLabeled['Sınav'][i-1] == df_gtsLabeled['Sınav'][i]:
            if df_gtsLabeled['Konu'][i-1] == df_gtsLabeled['Konu'][i]:
                if (df_gtsLabeled['S_Çıkış Zamanı'][i-1] > df_gtsLabeled['S_Giriş Zamanı'][i]) and (df_gtsLabeled['Puan'][i-1] < df_gtsLabeled['Puan'][i]) and (df_gtsLabeled['GirişLog'][i-1] == df_gtsLabeled['GirişLog'][i]):
                    df_gtsLabeled['Label'][i] = 1
                    df_gtsLabeled['Overlap (secs.)'][i] = (df_gtsLabeled['S_Giriş Zamanı'][i] - df_gtsLabeled['S_Çıkış Zamanı'][i-1]).total_seconds() * -1  # this is the overlapping time in seconds 


# name the table as df for the sake of simplicity
df = df_gtsLabeled

df

In [ ]:
# fill with zero if Label column is not 1

df.replace("", 0, inplace=True)

In [ ]:
df['Label'].value_counts()

# we have 7878 rows data and 290 of them are labeled as 1. This means that 290 members attempted gaming the system.

In [ ]:
# export df to csv file

df.to_csv('populated_df.csv', index=False)
